In [1]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.jit as jit
import torch.nn.functional as F
import numpy as np 
from torchvision import transforms
import random 

In [2]:
import numpy as np

batch_size = 32
seq_length = 10 
input_dim = 5

# Generate random time series data (e.g., random walk data)
np.random.seed(42)
data = np.cumsum(np.random.randn(batch_size, seq_length, input_dim), axis=1)

# Convert to PyTorch tensor
import torch
input_data = torch.tensor(data, dtype=torch.float32)
 
print(input_data.shape)  # Output: torch.Size([32, 10, 5])
input_dim = input_data.shape[2]
output_dim = input_data.shape[2]
output_seq_len = 5

torch.Size([32, 10, 5])


In [3]:
class Time2Vec(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.periodic = nn.Linear(input_dim, output_dim-1)
        
    def forward(self, x):
        linear_part = self.linear(x)
        periodic_part = torch.sin(self.periodic(x))
        return torch.cat([linear_part, periodic_part], dim=-1)

class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim)
        )
        
    def forward(self, x):
        return x + self.layers(x)

class FeaturePyramid(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.down1 = nn.Linear(input_dim, input_dim//2)
        self.down2 = nn.Linear(input_dim//2, input_dim//4)
        self.up1 = nn.Linear(input_dim//4, input_dim//2)
        self.up2 = nn.Linear(input_dim//2, input_dim)
        
    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        u1 = self.up1(d2)
        u2 = self.up2(u1 + d1)
        return u2 + x

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, output_seq_len):
        super().__init__()
        self.input_dim = input_dim
        self.output_seq_len = output_seq_len
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        
        # Create ModuleList to store layers
        self.layers = nn.ModuleList()
        
        # Conditional addition of positional encoding
        if random.random() < 0.5:
            self.pos_encoder = nn.Sequential(
                nn.Linear(1, 32),
                nn.ReLU(),
                nn.Linear(32, input_dim)
            )
            self.use_pos_encoding = True
        else:
            self.use_pos_encoding = False
            
        # Conditional addition of Time2Vec
        if random.random() < 0.5:
            self.time2vec = Time2Vec(input_dim, input_dim)
            self.use_time2vec = True
        else:
            self.use_time2vec = False
            
        # Conditional addition of multi-scale processing
        if random.random() < 0.5:
            self.scales = nn.ModuleList([
                nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, input_dim)
                ) for _ in range(3)
            ])
            self.use_multiscale = True
        else:
            self.use_multiscale = False
            
        # Base layers
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        self.layers.append(nn.ReLU())
        
        # Conditional addition of ResidualBlock
        if random.random() < 0.5:
            self.layers.append(ResidualBlock(hidden_dim))
            
        # Conditional addition of FeaturePyramid
        if random.random() < 0:
            self.layers.append(FeaturePyramid(hidden_dim))
        
        # Final output layer
        self.fc_last = nn.Linear(hidden_dim, output_dim * output_seq_len)

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        
        # Apply positional encoding if selected
        if self.use_pos_encoding:
            positions = torch.arange(seq_len, device=x.device).unsqueeze(-1).float()
            pos_encoding = self.pos_encoder(positions)
            x = x + pos_encoding.unsqueeze(0)
            
        # Apply Time2Vec if selected
        if self.use_time2vec:
            x = self.time2vec(x)
            
        # Flatten for processing
        x = x.view(-1, self.input_dim)
        
        # Apply multi-scale processing if selected
        if self.use_multiscale:
            scale_outputs = []
            for scale_layer in self.scales:
                scale_outputs.append(scale_layer(x))
            x = torch.mean(torch.stack(scale_outputs), dim=0)
        
        # Process through main layers
        for layer in self.layers:
            x = layer(x)
            
        # Final processing
        x = self.fc_last(x)
        x = x.view(batch_size, -1, self.output_dim)
        x = x[:, :self.output_seq_len, :]
        
        return x

# Usage remains the same
hidden_dim = 128
model = SimpleNN(input_dim, hidden_dim, output_dim, output_seq_len)
print(model)

# Define loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    
    for batch_data, batch_targets in input_data:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_data)  # (batch_size, output_seq_len, output_dim)
        
        # Compute loss
        loss = criterion(outputs, batch_targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


SimpleNN(
  (layers): ModuleList(
    (0): Linear(in_features=5, out_features=128, bias=True)
    (1): ReLU()
  )
  (pos_encoder): Sequential(
    (0): Linear(in_features=1, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=5, bias=True)
  )
  (time2vec): Time2Vec(
    (linear): Linear(in_features=5, out_features=1, bias=True)
    (periodic): Linear(in_features=5, out_features=4, bias=True)
  )
  (fc_last): Linear(in_features=128, out_features=25, bias=True)
)


ValueError: too many values to unpack (expected 2)